In [ ]:
# KObert 및 관련 라이브러리 설치
!pip install transformers tensorflow torch
# 추가적으로 필요한 라이브러리 설치 (pandas, numpy, scikit-learn 등)
!pip install pandas numpy scikit-learn

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("monologg/kobert")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [ ]:
!pip install sentencepiece
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")


In [ ]:
# ============================== #
# 1) 환경 설정 및 라이브러리 로드
# ============================== #
!pip -q install transformers==4.44.2 sentencepiece

import pandas as pd
import numpy as np
import random
import tensorflow as tf
import re

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from transformers import AutoTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense, Dropout, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model as KerasModel # Keras Model 클래스를 명확히 import


# ---- 하이퍼파라미터 ----
MAX_LEN = 128
EMBEDDING_DIM = 768
DROPOUT_RATE = 0.1
BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE = 5e-5
ANOMALY_WEIGHT_LAMBDA = 1.5
RANDOM_SEED = 42

# 재현성
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print("환경 설정 및 하이퍼파라미터 초기화 완료.")

# ============================== #
# 2) 데이터 로드 및 모의 라벨링
# ============================== #
file_path = '/content/yogiyo_reviews_30000.csv'
try:
    df = pd.read_csv(file_path)
    df = df[['content', 'score']].dropna()
    print(f"원본 데이터 로드 완료. 샘플 수: {len(df)}")
except FileNotFoundError:
    print(f"에러: 파일을 찾을 수 없습니다. {file_path}를 Colab에 업로드했는지 확인하세요.")
    print("샘플 데이터를 생성하여 진행합니다.")
    data = {
        'content': [f'샘플 리뷰 {i}입니다. 배달이 늦어 불만입니다.' if i % 5 == 0 else f'샘플 리뷰 {i}입니다.' for i in range(100)],
        'score': np.random.randint(1, 6, 100)
    }
    df = pd.DataFrame(data)
    df = df[['content', 'score']].dropna()

# ---- 감성 라벨링: 1~2=Neg, 3=Neu, 4~5=Pos ----
def label_sentiment(score):
    if score <= 2:
        return 'Negative'
    elif score == 3:
        return 'Neutral'
    else:
        return 'Positive'

df['sentiment_label'] = df['score'].apply(label_sentiment)

# ---- 요구사항 라벨 모의 생성 ----
REQUIREMENT_CATEGORIES = ['Delivery', 'UI/UX', 'Service', 'Price', 'Packaging']
def mock_label_requirements(text):
    keywords = {
        'Delivery': ['배달', '시간', '기사님', '지연', '늦어', '빨라'],
        'UI/UX': ['앱', '오류', '버그', '멈춰', '느려', '업데이트', '결제'],
        'Service': ['상담원', '고객센터', '응대', '친절', '불친절', '취소'],
        'Price': ['할인', '쿠폰', '배달비', '가격', '비싸', '요기패스'],
        'Packaging': ['포장', '새다', '흘러', '꼼꼼']
    }
    assigned = []
    text_lower = str(text).lower()
    for cat, kws in keywords.items():
        if any(kw in text_lower for kw in kws):
            assigned.append(cat)
    if not assigned:
        return random.choice(REQUIREMENT_CATEGORIES)
    return assigned[0]

df['requirement_label'] = df['content'].apply(mock_label_requirements)

# ---- 라벨 인코딩 ----
sentiment_map = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
requirement_map = {cat: i for i, cat in enumerate(REQUIREMENT_CATEGORIES)}

df['sentiment_encoded']   = df['sentiment_label'].map(sentiment_map)
df['requirement_encoded'] = df['requirement_label'].map(requirement_map)

print("데이터 로드 및 모의 라벨링 완료.")
display(df.head())

# ================================== #
# 3) KoBERT 토크나이저 인코딩 함수
# ================================== #
# 중요: skt/kobert-base-v1는 SentencePiece 기반 → AutoTokenizer 사용
ckpt = "skt/kobert-base-v1"
tokenizer = AutoTokenizer.from_pretrained(ckpt, use_fast=False)

def encode_texts(tokenizer, texts, max_len):
    # padding 방식 변경: pad_to_max_length (deprecated) → padding="max_length"
    enc = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=max_len,
        return_tensors="tf"
    )
    return enc["input_ids"], enc["attention_mask"]

X_input_ids, X_attention_masks = encode_texts(tokenizer, df['content'].tolist(), MAX_LEN)
print(f"\n인코딩된 입력 (input_ids) 형태: {X_input_ids.shape}")
print(f"인코딩된 입력 (attention_masks) 형태: {X_attention_masks.shape}")

# ================================== #
# 4) 레이블/마스크 및 데이터 분할
# ================================== #
Y_sentiment   = to_categorical(df['sentiment_encoded'].values,   num_classes=len(sentiment_map))
Y_requirement = to_categorical(df['requirement_encoded'].values, num_classes=len(requirement_map))

# 이상치 마스크: Negative(0)일 때 1, 아니면 0
Y_anomaly_mask = np.where(df['sentiment_encoded'].values == 0, 1.0, 0.0)
print(f"\n전체 리뷰 중 Negative(Anomaly) 비율: {Y_anomaly_mask.mean():.2f}")

# Convert TensorFlow tensors to NumPy arrays before splitting
X_input_ids_np = X_input_ids.numpy()
X_attention_masks_np = X_attention_masks.numpy()

X_train_ids, X_test_ids, X_train_masks, X_test_masks, \
YS_train, YS_test, YR_train, YR_test, YM_train, YM_test = train_test_split(
    X_input_ids_np, X_attention_masks_np, Y_sentiment, Y_requirement, Y_anomaly_mask,
    test_size=0.2, random_state=RANDOM_SEED
)

print(f"\n학습 데이터 샘플 수: {len(X_train_ids)}, 검증 데이터 샘플 수: {len(X_test_ids)}")
print("데이터 로드, 전처리 및 분할 완료. KObert 모델 구축 준비 완료.")

# ================================== #
# 5) 모델 정의 (TFBertModel + 2헤드 - Subclassing)
# ================================== #
class KobertDualOutputModel(KerasModel):
    def __init__(self, bert_model_name, num_sentiment_classes, num_requirement_classes, dropout_rate, **kwargs):
        super().__init__(**kwargs)
        # 주의: KoBERT는 TF 가중치가 없으므로 from_pt=True 필요
        self.bert = TFBertModel.from_pretrained(bert_model_name, from_pt=True)
        self.dropout = Dropout(dropout_rate)
        self.sentiment_classifier = Dense(num_sentiment_classes, activation='softmax', name="sentiment")
        self.requirement_classifier = Dense(num_requirement_classes, activation='softmax', name="requirement")

    def call(self, inputs, training=False):
        # inputs는 딕셔너리 형태를 예상: {'input_ids': ..., 'attention_mask': ...}
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']

        # BERT 모델 통과
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, training=training)

        # 보편적으로 [CLS] 토큰 벡터 사용 (first token)
        pooled_output = bert_outputs.last_hidden_state[:, 0, :]

        # Dropout 적용
        x = self.dropout(pooled_output, training=training)

        # 분류 헤드 통과
        sentiment_logits = self.sentiment_classifier(x)
        requirement_logits = self.requirement_classifier(x)

        return {"sentiment": sentiment_logits, "requirement": requirement_logits}

# 모델 인스턴스 생성
model = KobertDualOutputModel(
    bert_model_name=ckpt,
    num_sentiment_classes=len(sentiment_map),
    num_requirement_classes=len(requirement_map),
    dropout_rate=DROPOUT_RATE
)

# ================================== #
# 6) 손실/옵티마이저/지표 설정
# ================================== #
# 기본 크로스엔트로피
losses = {
    "sentiment":   tf.keras.losses.CategoricalCrossentropy(),
    "requirement": tf.keras.losses.CategoricalCrossentropy()
}

metrics = {
    "sentiment":   [tf.keras.metrics.CategoricalAccuracy(name="acc")],
    "requirement": [tf.keras.metrics.CategoricalAccuracy(name="acc")]
}

optimizer = Adam(learning_rate=LEARNING_RATE)

# Subclassing 모델은 build() 메서드를 호출하여 입력 형태를 명시해주거나, 첫 번째 fit/evaluate 호출 시 자동 빌드됨
# 명시적으로 build 해주는 것이 좋습니다.
model.build(input_shape={"input_ids": (None, MAX_LEN), "attention_mask": (None, MAX_LEN)})


model.compile(optimizer=optimizer, loss=losses, metrics=metrics)
model.summary()

# ================================== #
# 7) 샘플 가중치(이상치 가중) 설정
# ================================== #
# requirement 쪽에만 이상치 가중치 적용:
# sample_weight_req = 1 + (lambda-1) * anomaly_mask
sample_weight_req_train = 1.0 + (ANOMALY_WEIGHT_LAMBDA - 1.0) * YM_train
sample_weight_req_test  = 1.0 + (ANOMALY_WEIGHT_LAMBDA - 1.0) * YM_test

# sentiment 쪽은 균등 가중치(전부 1.0)
sample_weight_sent_train = np.ones_like(YM_train, dtype=np.float32)
sample_weight_sent_test  = np.ones_like(YM_test, dtype=np.float32)

# Keras는 출력별 sample_weight를 dict로 받는다.
train_sample_weights = {
    "sentiment":   sample_weight_sent_train,
    "requirement": sample_weight_req_train
}
val_sample_weights = {
    "sentiment":   sample_weight_sent_test,
    "requirement": sample_weight_req_test
}

# ================================== #
# 8) 학습
# ================================== #
print("\n모델 학습 시작...")
history = model.fit(
    x={"input_ids": X_train_ids, "attention_mask": X_train_masks},
    y={"sentiment": YS_train, "requirement": YR_train},
    sample_weight=train_sample_weights,
    validation_data=(
        {"input_ids": X_test_ids, "attention_mask": X_test_masks},
        {"sentiment": YS_test, "requirement": YR_test},
        val_sample_weights
    ),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

print("\n모델 학습 완료.")

# ================================== #
# 9) 평가 및 간단 예측 예시
# ================================== #
eval_result = model.evaluate(
    x={"input_ids": X_test_ids, "attention_mask": X_test_masks},
    y={"sentiment": YS_test, "requirement": YR_test},
    sample_weight=val_sample_weights,
    batch_size=BATCH_SIZE,
    return_dict=True
)
print("\n[평가 결과]")
for k, v in eval_result.items():
    print(f"{k}: {v:.4f}")

# 임의 문장 예측
sample_texts = [
    "배달이 너무 늦어서 화가 났습니다.",
    "앱 결제가 자꾸 오류가 나요.",
    "가격도 괜찮고 포장도 깔끔했어요."
]
sample_ids, sample_masks = encode_texts(tokenizer, sample_texts, MAX_LEN)
pred = model.predict({"input_ids": sample_ids, "attention_mask": sample_masks})

inv_sent = {v:k for k,v in sentiment_map.items()}
inv_req  = {v:k for k,v in requirement_map.items()}

print("\n[샘플 예측]")
for t, ps, pr in zip(sample_texts, pred["sentiment"], pred["requirement"]):
    s_cls = inv_sent[int(np.argmax(ps))]
    r_cls = inv_req[int(np.argmax(pr))]
    print(f"- \"{t}\": Sentiment={s_cls}, Requirement={r_cls}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 61.2 MB/s eta 0:00:00
환경 설정 및 하이퍼파라미터 초기화 완료.
원본 데이터 로드 완료. 샘플 수: 30000
데이터 로드 및 모의 라벨링 완료.


,content,score,sentiment_label,requirement_label,sentiment_encoded,requirement_encoded
0,파란녀석이랑 다르게 수저포크 다시 물어봐주는거 너무좋아요,5,Positive,Delivery,2,0
1,요기요 배달기사님들 제발 벨좀 눌러주세요.... 라이더 요청사항 매번 적으면 뭐합니...,1,Negative,Delivery,0,0
2,타 경쟁사보다 쿠폰을 짱 많이줘서 좋아요^^,5,Positive,Price,2,3
3,배민보다는.. 요기요가!!!짱,5,Positive,Delivery,2,0
4,좋아요,5,Positive,Service,2,2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sentiment (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ requirement (Dense)             │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


모델 학습 시작...
Epoch 1/5
226/750 ━━━━━━━━━━━━━━━━━━━━ 2:40:39 18s/step - loss: 2.7195 - requirement_acc: 0.2559 - requirement_loss: 1.7861 - sentiment_acc: 0.6065 - sentiment_loss: 0.9334

In [ ]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import re

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# KObert를 위한 AutoTokenizer 및 TFBertModel import
from transformers import AutoTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense, Dropout, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model as KerasModel # Keras Model 클래스를 명확히 import


# ---- 하이퍼파라미터 ----
# 논문 3.5.2절 기반 - KObert에 맞게 일부 조정
MAX_LEN = 128         # KObert는 일반적으로 더 긴 시퀀스 길이를 사용
EMBEDDING_DIM = 768   # KObert base 모델의 임베딩 차원
DROPOUT_RATE = 0.1    # KObert fine-tuning 시 흔히 사용되는 dropout 비율
BATCH_SIZE = 32       # GPU 메모리 고려하여 batch size 조정
EPOCHS = 3            # Fine-tuning 시에는 적은 epoch으로 충분할 수 있습니다.
LEARNING_RATE = 5e-5  # KObert fine-tuning을 위한 학습률
ANOMALY_WEIGHT_LAMBDA = 1.5 # 이상치 가중치 (논문 3.4.2절 기반)
RANDOM_SEED = 42

# 재현성
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print("환경 설정 및 하이퍼파라미터 초기화 완료.")

# ============================== #
# 2) 데이터 로드 및 모의 라벨링
# ============================== #
file_path = '/content/yogiyo_reviews_5000.csv'
try:
    df = pd.read_csv(file_path)
    df = df[['content', 'score']].dropna()
    print(f"원본 데이터 로드 완료. 샘플 수: {len(df)}")
except FileNotFoundError:
    print(f"에러: 파일을 찾을 수 없습니다. {file_path}를 Colab에 업로드했는지 확인하세요.")
    print("샘플 데이터를 생성하여 진행합니다.")
    data = {
        'content': [f'샘플 리뷰 {i}입니다. 배달이 늦어 불만입니다.' if i % 5 == 0 else f'샘플 리뷰 {i}입니다.' for i in range(100)],
        'score': np.random.randint(1, 6, 100)
    }
    df = pd.DataFrame(data)
    df = df[['content', 'score']].dropna()

# ---- 감성 라벨링: 1~2=Neg, 3=Neu, 4~5=Pos ----
def label_sentiment(score):
    if score <= 2:
        return 'Negative'
    elif score == 3:
        return 'Neutral'
    else:
        return 'Positive'

df['sentiment_label'] = df['score'].apply(label_sentiment)

# ---- 요구사항 라벨 모의 생성 ----
REQUIREMENT_CATEGORIES = ['Delivery', 'UI/UX', 'Service', 'Price', 'Packaging']
def mock_label_requirements(text):
    keywords = {
        'Delivery': ['배달', '시간', '기사님', '지연', '늦어', '빨라'],
        'UI/UX': ['앱', '오류', '버그', '멈춰', '느려', '업데이트', '결제'],
        'Service': ['상담원', '고객센터', '응대', '친절', '불친절', '취소'],
        'Price': ['할인', '쿠폰', '배달비', '가격', '비싸', '요기패스'],
        'Packaging': ['포장', '새다', '흘러', '꼼꼼']
    }
    assigned = []
    text_lower = str(text).lower()
    for cat, kws in keywords.items():
        if any(kw in text_lower for kw in kws):
            assigned.append(cat)
    if not assigned:
        # 할당된 라벨이 없는 경우 가장 흔한 카테고리 중 하나를 랜덤으로 할당 (노이즈 방지)
        # 실제 데이터에서는 '기타' 카테고리 등으로 처리하거나 라벨링 규칙을 보완해야 합니다.
        return random.choice(REQUIREMENT_CATEGORIES)
    # 여러 라벨이 할당될 경우 첫 번째 라벨만 사용 (단일 출력 가정)
    return assigned[0]

df['requirement_label'] = df['content'].apply(mock_label_requirements)

# ---- 라벨 인코딩 ----
sentiment_map = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
requirement_map = {cat: i for i, cat in enumerate(REQUIREMENT_CATEGORIES)}

df['sentiment_encoded']   = df['sentiment_label'].map(sentiment_map)
df['requirement_encoded'] = df['requirement_label'].map(requirement_map)

print("데이터 로드 및 모의 라벨링 완료.")
display(df.head())

# ================================== #
# 3) KoBERT 토크나이저 인코딩 함수
# ================================== #
# 중요: skt/kobert-base-v1는 SentencePiece 기반 → AutoTokenizer 사용
ckpt = "skt/kobert-base-v1"
tokenizer = AutoTokenizer.from_pretrained(ckpt, use_fast=False)


def encode_texts(tokenizer, texts, max_len):
    enc = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=max_len,
        return_tensors="tf" # TensorFlow 텐서 반환
    )
    return enc["input_ids"], enc["attention_mask"]

X_input_ids, X_attention_masks = encode_texts(tokenizer, df['content'].tolist(), MAX_LEN)
print(f"\n인코딩된 입력 (input_ids) 형태: {X_input_ids.shape}")
print(f"인코딩된 입력 (attention_masks) 형태: {X_attention_masks.shape}")

# ================================== #
# 4) 레이블/마스크 및 데이터 분할
# ================================== #
Y_sentiment   = to_categorical(df['sentiment_encoded'].values,   num_classes=len(sentiment_map))
Y_requirement = to_categorical(df['requirement_encoded'].values, num_classes=len(requirement_map))

# 이상치 마스크: Negative(0)일 때 1, 아니면 0
Y_anomaly_mask = np.where(df['sentiment_encoded'].values == 0, 1.0, 0.0)
print(f"\n전체 리뷰 중 Negative(Anomaly) 비율: {Y_anomaly_mask.mean():.2f}")

# TensorFlow 텐서를 NumPy 배열로 변환하여 sklearn의 train_test_split에 전달
X_input_ids_np = X_input_ids.numpy()
X_attention_masks_np = X_attention_masks.numpy()
Y_sentiment_np = Y_sentiment
Y_requirement_np = Y_requirement
Y_anomaly_mask_np = Y_anomaly_mask


X_train_ids, X_test_ids, X_train_masks, X_test_masks, \
YS_train, YS_test, YR_train, YR_test, YM_train, YM_test = train_test_split(
    X_input_ids_np, X_attention_masks_np, Y_sentiment_np, Y_requirement_np, Y_anomaly_mask_np,
    test_size=0.2, random_state=RANDOM_SEED
)

print(f"\n학습 데이터 샘플 수: {len(X_train_ids)}, 검증 데이터 샘플 수: {len(X_test_ids)}")
print("데이터 로드, 전처리 및 분할 완료. KObert 모델 구축 준비 완료.")

# ================================== #
# 5) 모델 정의 (TFBertModel + 2헤드 - Subclassing)
# ================================== #
class KobertDualOutputModel(KerasModel):
    def __init__(self, bert_model_name, num_sentiment_classes, num_requirement_classes, dropout_rate, **kwargs):
        super().__init__(**kwargs)
        # 주의: KoBERT는 TF 가중치가 없으므로 from_pt=True 필요
        self.bert = TFBertModel.from_pretrained(bert_model_name, from_pt=True)
        self.dropout = Dropout(dropout_rate)
        self.sentiment_classifier = Dense(num_sentiment_classes, activation='softmax', name="sentiment")
        self.requirement_classifier = Dense(num_requirement_classes, activation='softmax', name="requirement")

    def call(self, inputs, training=False):
        # inputs는 딕셔너리 형태를 예상: {'input_ids': ..., 'attention_mask': ...}
        # TF 텐서로 입력이 들어옴
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']

        # BERT 모델 통과
        # output_attentions=False, output_hidden_states=False 는 기본값
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, training=training)

        # 보편적으로 [CLS] 토큰 벡터 사용 (first token)
        pooled_output = bert_outputs.last_hidden_state[:, 0, :]

        # Dropout 적용
        x = self.dropout(pooled_output, training=training)

        # 분류 헤드 통과
        sentiment_logits = self.sentiment_classifier(x)
        requirement_logits = self.requirement_classifier(x)

        return {"sentiment": sentiment_logits, "requirement": requirement_logits}

# 모델 인스턴스 생성
model = KobertDualOutputModel(
    bert_model_name=ckpt,
    num_sentiment_classes=len(sentiment_map),
    num_requirement_classes=len(requirement_map),
    dropout_rate=DROPOUT_RATE
)

# ================================== #
# 6) 손실/옵티마이저/지표 설정
# ================================== #
# 기본 크로스엔트로피
losses = {
    "sentiment":   tf.keras.losses.CategoricalCrossentropy(),
    "requirement": tf.keras.losses.CategoricalCrossentropy()
}

metrics = {
    "sentiment":   [tf.keras.metrics.CategoricalAccuracy(name="acc")],
    "requirement": [tf.keras.metrics.CategoricalAccuracy(name="acc")]
}

optimizer = Adam(learning_rate=LEARNING_RATE)

# Subclassing 모델은 build() 메서드를 호출하여 입력 형태를 명시해주거나, 첫 번째 fit/evaluate 호출 시 자동 빌드됨
# 명시적으로 build 해주는 것이 좋습니다.
model.build(input_shape={"input_ids": (None, MAX_LEN), "attention_mask": (None, MAX_LEN)})


model.compile(optimizer=optimizer, loss=losses, metrics=metrics)

# summary 호출 전에 build가 되어야 제대로 출력됨
model.summary()


# ================================== #
# 7) 샘플 가중치(이상치 가중) 설정
# ================================== #
# requirement 쪽에만 이상치 가중치 적용:
# sample_weight_req = 1 + (lambda-1) * anomaly_mask
sample_weight_req_train = 1.0 + (ANOMALY_WEIGHT_LAMBDA - 1.0) * YM_train
sample_weight_req_test  = 1.0 + (ANOMALY_WEIGHT_LAMBDA - 1.0) * YM_test

# sentiment 쪽은 균등 가중치(전부 1.0)
sample_weight_sent_train = np.ones_like(YM_train, dtype=np.float32)
sample_weight_sent_test  = np.ones_like(YM_test, dtype=np.float32)

# Keras는 출력별 sample_weight를 dict로 받는다.
train_sample_weights = {
    "sentiment":   sample_weight_sent_train,
    "requirement": sample_weight_req_train
}
val_sample_weights = {
    "sentiment":   sample_weight_sent_test,
    "requirement": sample_weight_req_test
}

# ================================== #
# 8) 학습
# ================================== #
print("\n모델 학습 시작...")
history = model.fit(
    x={"input_ids": X_train_ids, "attention_mask": X_train_masks},
    y={"sentiment": YS_train, "requirement": YR_train},
    sample_weight=train_sample_weights,
    validation_data=(
        {"input_ids": X_test_ids, "attention_mask": X_test_masks},
        {"sentiment": YS_test, "requirement": YR_test},
        val_sample_weights
    ),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

print("\n모델 학습 완료.")

# ================================== #
# 9) 평가 및 간단 예측 예시
# ================================== #
print("\n[평가 결과]")
eval_result = model.evaluate(
    x={"input_ids": X_test_ids, "attention_mask": X_test_masks},
    y={"sentiment": YS_test, "requirement": YR_test},
    sample_weight=val_sample_weights,
    batch_size=BATCH_SIZE,
    return_dict=True
)
for k, v in eval_result.items():
    print(f"{k}: {v:.4f}")

# 임의 문장 예측
sample_texts = [
    "배달이 너무 늦어서 화가 났습니다.",
    "앱 결제가 자꾸 오류가 나요.",
    "가격도 괜찮고 포장도 깔끔했어요."
]
# encode_texts 함수는 TF 텐서를 반환하므로 .numpy() 변환 불필요
sample_ids, sample_masks = encode_texts(tokenizer, sample_texts, MAX_LEN)

pred = model.predict({"input_ids": sample_ids, "attention_mask": sample_masks})

inv_sent = {v:k for k,v in sentiment_map.items()}
inv_req  = {v:k for k,v in requirement_map.items()}

print("\n[샘플 예측]")
for t, ps, pr in zip(sample_texts, pred["sentiment"], pred["requirement"]):
    s_cls = inv_sent[int(np.argmax(ps))]
    r_cls = inv_req[int(np.argmax(pr))]
    print(f"- \"{t}\": Sentiment={s_cls}, Requirement={r_cls}")

환경 설정 및 하이퍼파라미터 초기화 완료.
원본 데이터 로드 완료. 샘플 수: 5000
데이터 로드 및 모의 라벨링 완료.


,content,score,sentiment_label,requirement_label,sentiment_encoded,requirement_encoded
0,전체적으로 맛있어요,5,Positive,Delivery,2,0
1,배달앱중에 제일 하급 배달 겁나 느리고 배달예정시간 한없이 늘어나고 보상도없음 라이...,1,Negative,Delivery,0,0
2,오배송되서 고객센터 전화했는데 상담원 연결 자체가 없음 개 쓰레기임 환불받으면 삭제...,1,Negative,Service,0,2
3,ㅇㄱㅇ요기요,5,Positive,Delivery,2,0
4,아이를 간식 시킬땐 여기요 요기요,5,Positive,Service,2,2


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



인코딩된 입력 (input_ids) 형태: (5000, 128)
인코딩된 입력 (attention_masks) 형태: (5000, 128)

전체 리뷰 중 Negative(Anomaly) 비율: 0.25

학습 데이터 샘플 수: 4000, 검증 데이터 샘플 수: 1000
데이터 로드, 전처리 및 분할 완료. KObert 모델 구축 준비 완료.


All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'kobert_dual_output_model_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sentiment (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ requirement (Dense)             │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


모델 학습 시작...
Epoch 1/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 1987s 16s/step - loss: 3.0178 - requirement_acc: 0.1762 - requirement_loss: 1.9444 - sentiment_acc: 0.4089 - sentiment_loss: 1.0733 - val_loss: 2.6393 - val_requirement_acc: 0.3020 - val_requirement_loss: 1.7972 - val_sentiment_acc: 0.6670 - val_sentiment_loss: 0.8396
Epoch 2/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 1994s 16s/step - loss: 2.5793 - requirement_acc: 0.2968 - requirement_loss: 1.7756 - sentiment_acc: 0.6821 - sentiment_loss: 0.8037 - val_loss: 2.4251 - val_requirement_acc: 0.3380 - val_requirement_loss: 1.7260 - val_sentiment_acc: 0.6960 - val_sentiment_loss: 0.7002
Epoch 3/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 1982s 16s/step - loss: 2.4008 - requirement_acc: 0.3229 - requirement_loss: 1.7144 - sentiment_acc: 0.7325 - sentiment_loss: 0.6864 - val_loss: 2.3249 - val_requirement_acc: 0.3460 - val_requirement_loss: 1.7000 - val_sentiment_acc: 0.7060 - val_sentiment_loss: 0.6281

모델 학습 완료.

[평가 결과]
32/32 ━━━━━━━━━━━━━━━━━━━━ 367s 11s/step - 